In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import fiona
from dbfread import DBF
from backend.geotools.path_calc import address_search
import seaborn as sns


In [ ]:
d = Path("../webapp/backend/backend/data/AADT_1997_2019_History_Table_AccessDB/AADT_1997-2019.csv")
historical = pd.read_csv(d)
historical.columns

In [ ]:
tables = DBF("../webapp/backend/backend/data/traffic/Annual_Average_Daily_Traffic_Locations_in_Minnesota.dbf", load=True)
gis = pd.DataFrame(tables.records)

In [ ]:
traffic = historical.merge(gis[["SEQUENCE_N","LONGITUDE","LATITUDE"]],left_on="SEQUENCE_NUMBER", right_on="SEQUENCE_N")

In [ ]:
keep_cols = [col for col in traffic.columns if "DATA_YEAR" not in col or "AADT" not in col]
def min_max_difference(df):
    years = df["DATA_YEAR"].values
    min_year = np.min(years)
    max_year = np.max(years)
    min_year_traffic = df.loc[years == min_year, "AADT"].min()
    max_year_traffic = df.loc[years == max_year, "AADT"].max()
    difference = max_year_traffic - min_year_traffic
    df = df.loc[:, keep_cols].drop_duplicates().iloc[0]
    df["MIN"] = min_year_traffic
    df["MAX"] = max_year_traffic
    df["DIFF"] = difference
    df["MIN_YEAR"] = min_year
    df["MAX_YEAR"] = max_year
    return df


In [ ]:

traffic_changes = traffic.groupby("SEQUENCE_NUMBER").apply(min_max_difference)
# traffic_changes = pd.read_csv("data/Diff_Data.csv")

In [ ]:
traffic_changes.to_csv("data/Diff_Data.csv").reset_index(drop=True)

In [ ]:
traffic_changes["DIFF"].idxmax()

In [ ]:
traffic_changes.iloc[3716]

In [ ]:
seq = traffic_changes.iloc[3716]["SEQUENCE_NUMBER"]
seq

In [ ]:
seq = traffic_changes.iloc[33280]["SEQUENCE_NUMBER"]

In [ ]:
traffic = traffic.sort_values(["SEQUENCE_NUMBER", "DATA_YEAR"])

In [ ]:
traffic.loc[traffic["SEQUENCE_NUMBER"] == seq].plot(x="DATA_YEAR", y="AADT")

In [ ]:
sns.displot(x="DIFF", data=traffic_changes)